In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
sys.path.append(rf'/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest')
from PointDataLoader import PointDataLoader

In [3]:
import pickle 
with open('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/data_for_weather_kriging.pkl', 'rb') as f:
     loaded_dict = pickle.load(f)

ignitions = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_negative_weather_grid.geojson')

In [ ]:
d = ignitions[ignitions.YYYY == 2016]
# f = loaded_dict[2016]['rtot']
meteo = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/database/meteo_2016.geojson')

In [10]:
for i in tqdm(d.index):
    P = PointDataLoader(anno = d.YYYY.loc[i], mese = d.MM.loc[i], giorno = d.DD.loc[i], x = d.geometry.loc[i].x, y = d.geometry.loc[i].y, crs=d.crs)
    P.load()

 36%|███▋      | 43/118 [36:47<1:04:09, 51.33s/it]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [26]:
row = d.iloc[43]

In [27]:
row

YYYY                                              2016
day                                                335
height                                          2201.0
DD                                                  30
MM                                                  10
geometry    POINT (773225.306140101 5620903.479513713)
Name: 371, dtype: object

In [28]:
aux=meteo
pd.Series(aux[aux.day == row.day].loc[aux[aux.day == row.day].index[0],[col for col in aux.columns if col not in ['geometry', 'day', 'YYYY', 'height']]])

tmedia                  0.038915
tmax                    6.543246
tmin                   -6.230928
ptot                    0.052354
vmedia                  2.062415
vraffica                6.572014
settore_prevalente     11.013124
tempo_permanenza      754.384098
durata_calma           23.416963
umedia                 19.698237
umin                   11.693637
umax                   34.120215
rtot                    7.486227
hdd_base18             16.857544
hdd_base20             18.857544
cdd_base18                   0.0
Name: 66, dtype: object

---

In [ ]:
target = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_negative_weather_grid.geojson')

In [28]:
def reverse_day(year, day):
    feb = 28
    if year in list(range(2000, 2025, 4)):
        feb += 1
            
    days_per_month = [31, feb, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    for index, month in enumerate(days_per_month):
        if (day - month) == 0:
            return (month, index)
        elif (day - month) > 0:
            day -= month
        else:
            return (day, index+1)


In [32]:
target['DD'] = target.apply(lambda x: reverse_day(x['YYYY'], x['day'])[0], axis=1)
target['MM'] = target.apply(lambda x: reverse_day(x['YYYY'], x['day'])[1], axis=1)

In [34]:
target.to_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_negative_weather_grid.geojson', index=False)

---

In [ ]:
dic = {'Pianura': 0,
       'Collina': 1,
       'Montagna': 2}
gdf['zalt'] = gdf['zalt_des'].map(dic)

In [ ]:
gdf.drop(columns=['cod_istat', 'zalt_des'], inplace=True)
gdf.rename(columns={'zalt': 'zona_altimetrica'}, inplace=True)

In [ ]:
dic = dict.fromkeys([col for col in gdf.columns if col not in ['geometry']])
for a in dic:
    dic[a] = 'ALTIMETRIA_'+a

gdf.rename(columns=dic, inplace=True)

In [ ]:
gdf.to_file('/nfs/home/genovese/thesis-wildfire-genovese/database/altimetria.geojson', driver='GeoJSON', index=False)

---

In [ ]:
mising = df[df.height.isna()]
# 45 points

In [ ]:
# got from https://twcc.fr/
fill = {1: 1580,
 125: 569,
 131: 1049,
 146: 1447,
 165: 1345,
 217: 1530,
 225: 1339,
 227: 1361,
 230: 1389,
 258: 563,
 274: 614,
 351: 248,
 383: 642,
 394: 725,
 396: 1298,
 397: 1536,
 404: 766,
 438: 1093,
 440: 1034,
 443: 1356,
 448: 1053,
 458: 1494,
 460: 1298,
 461: 1029,
 468: 559,
 476: 1428,
 477: 1769,
 482: 1538,
 484: 1436,
 501: 406,
 504: 891,
 634: 344,
 635: 373,
 760: 428,
 772: 891,
 826: 1039,
 866: 533,
 996: 1068,
 1011: 568,
 1012: 568,
 1025: 812,
 1035: 585,
 1148: 742,
 1174: 1128,
 1206: 923}

In [ ]:
df['height'].fillna(fill, inplace=True)

In [ ]:
save_clean_data(df, 'kriging_weather_grid', '/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging' )